In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
RS = sum(list(map(ord, 'Dale Boca')))

import pandas as pd
pd.option_context('display.max_rows', None, 'display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

import numpy as np
import os;

## Lectura y Preparación del Dataset

In [2]:
data = pd.read_csv('dataset_final.csv', index_col=0)

In [3]:
from sklearn.preprocessing import LabelEncoder

def data_preparation(df, window=3, neutral=True):
    
    # armar las windows para el armado de las features 
    cols1 = ['pos', 'neg', 'neu'] if neutral==True else ['pos', 'neg']
    for col in cols1:
        df[col+'w'] = df[col].rolling(window=window).mean()
    
    # shifteo un día el retorno. las noticias deben estimar el retorno del día posterior.
    df['exc_ret']  = df['exc_ret'].shift(periods=-1)
    df['exc_ret%'] = df['exc_ret%'].shift(periods=-1) # por si lo necesito 
    
    # elimino las filas que quedan con nan por efecto del rolling window
    df.dropna(axis=0, inplace=True)
    
    # encoding del label que indica si hay retorno positivo (1) o negativo (0)
    #le = LabelEncoder()
    #df['exc_ret'] = le.fit_transform(df['exc_ret'])
    
    # split en train y test por fecha
    cols2 = [x+'w' for x in cols1]
    
    cut_date = '2018-10-31'
    
    X  = df.loc[df.date <= cut_date, cols2]
    y  = df.loc[df.date <= cut_date, 'exc_ret%']
    
    Xt = df.loc[df.date > cut_date, cols2]
    yt = df.loc[df.date > cut_date, 'exc_ret%']
    
    return X, Xt, y, yt

## Entrenamiento de Regressors

In [4]:
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import roc_auc_score, r2_score, mean_squared_error

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost.sklearn import XGBClassifier, XGBRegressor
import lightgbm as lgb
#from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.svm import SVC, SVR

In [5]:
def train_regressor(X, Xt, y, yt):
    
    #model_name = ['Random Forest', 'XGBoost', 'LightGBM', 'Logistic Regression',
    #              'Catboost', 'Support Vector']
    model_name = ['Random Forest', 'XGBoost', 'LightGBM', 'Catboost', 'Support Vector']
   
    model_init = [RandomForestRegressor(),
                  XGBRegressor(),
                  lgb.LGBMRegressor(),
                  #LogisticRegression(),
                  CatBoostRegressor(),
                  SVR()
                 ]
    
    # Random Forest 
    params1 = {  
        "n_estimators": st.randint(10,100),    # Number of boosted trees to fit.
        "max_depth"   : st.randint(2, 25),     # Maximum tree depth for base learners.
    }
    
    # XGBoost y Lightgbm
    params2 = {  
        "n_estimators": st.randint(10,100),    # Number of boosted trees to fit.
        "max_depth": st.randint(2, 25),         # Maximum tree depth for base learners.
        "learning_rate": st.uniform(0.01, 0.5), # Boosting learning rate (xgb’s “eta”)
        "colsample_bytree": st.beta(10, 1),     # Subsample ratio of columns when constructing each tree.
        "subsample": st.beta(10, 1),            # Subsample ratio of the training instance.
        "gamma": st.uniform(0, 10),             # Minimum loss reduction required to make a further partition on a leaf node of the tree.
        'reg_alpha': st.uniform(0.05,10),       # L1 regularization term on weights
        "min_child_weight": st.uniform(1,20),   # Minimum sum of instance weight(hessian) needed in a child.
    }
    
    # Logistic Regression
    params3 = {  
        "penalty"     : ['l1', 'l2'],
        "C"           : st.uniform(0.1, 10.),
    }
    
    # Catboost
    params4 = {}
    
    # SVC
    params5 = {
        "C"           : st.uniform(0.1, 10.),
        #"kernel"      : ['lbf', 'linear']
    }
    
    #model_params = [params1, params2, params2, params3, params4, params5]
    model_params = [params1, params2, params2, params4, params5]
    
    train_scores    = []
    test_scores     = []
    best_estimators = []
    best_parameters = []
    
    #kf    = KFold(n_splits=5, shuffle=True, random_state=RS)
    tsp   = TimeSeriesSplit(n_splits=6)
    
    for name, mod, params in zip(model_name, model_init, model_params):
        
        model = mod

        rgrid = RandomizedSearchCV(estimator=model, param_distributions=params, cv=tsp,
                                   scoring='neg_mean_squared_error', n_iter=10,
                                   verbose=1, n_jobs=-1, error_score=0)
        rgrid.fit(X, y)
        
        best_estimators.append(rgrid.best_estimator_)
        best_parameters.append(rgrid.best_params_)
        train_scores.append(rgrid.best_score_)
        
        ypred = rgrid.predict(Xt)
        score = r2_score(y_true=yt, y_pred=ypred)
        
        test_scores.append(score)
    
    results = pd.DataFrame(
        {'Model'      : model_name,
         'Train Score': train_scores,
         'Test Score' : test_scores,
         'Params'     : best_parameters,
         'Estimator'  : best_estimators
        })
    
    return results

In [6]:
import itertools
windows = np.arange(1, 6)
neutral = [1, 0]

resultado = pd.DataFrame()

for win, neu in itertools.product(windows, neutral):
    
    X, X_test, y, y_test = data_preparation(data, window=win, neutral=neu)
    
    regr = train_regressor(X, X_test, y, y_test)
  
    res = pd.DataFrame(
    {'Model'      : regr['Model'],
     'Window'     : list(str(win)*regr.shape[0]),
     'Neutral'    : list(str(neu)*regr.shape[0]),
     'Train Score': regr['Train Score'],
     'Test Score' : regr['Test Score'],
     'Params'     : regr['Params'],
     #'Estimator'  : regr['Estimator']
    })
    
    resultado = pd.concat([resultado, res], axis=0)

Fitting 6 folds for each of 10 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    3.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 6 folds for each of 10 candidates, totalling 60 fits
Fitting 6 folds for each of 10 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 6 folds for each of 1 candidates, totalling 6 fits


...

210:	learn: 0.0146370	total: 564ms	remaining: 2.11s
211:	learn: 0.0146306	total: 568ms	remaining: 2.11s
212:	learn: 0.0146229	total: 572ms	remaining: 2.11s
213:	learn: 0.0146137	total: 575ms	remaining: 2.11s
214:	learn: 0.0146092	total: 580ms	remaining: 2.12s
215:	learn: 0.0146057	total: 583ms	remaining: 2.12s
216:	learn: 0.0145925	total: 587ms	remaining: 2.12s
217:	learn: 0.0145890	total: 591ms	remaining: 2.12s
218:	learn: 0.0145838	total: 596ms	remaining: 2.13s
219:	learn: 0.0145746	total: 600ms	remaining: 2.13s
220:	learn: 0.0145654	total: 604ms	remaining: 2.13s
221:	learn: 0.0145506	total: 609ms	remaining: 2.13s
222:	learn: 0.0145364	total: 612ms	remaining: 2.13s
223:	learn: 0.0145333	total: 616ms	remaining: 2.13s
224:	learn: 0.0145283	total: 620ms	remaining: 2.14s
225:	learn: 0.0145166	total: 625ms	remaining: 2.14s
226:	learn: 0.0145139	total: 626ms	remaining: 2.13s
227:	learn: 0.0145104	total: 629ms	remaining: 2.13s
228:	learn: 0.0145069	total: 632ms	remaining: 2.13s
229:	learn: 

415:	learn: 0.0135119	total: 1.12s	remaining: 1.57s
416:	learn: 0.0135098	total: 1.13s	remaining: 1.57s
417:	learn: 0.0135032	total: 1.13s	remaining: 1.57s
418:	learn: 0.0135011	total: 1.13s	remaining: 1.57s
419:	learn: 0.0135006	total: 1.14s	remaining: 1.57s
420:	learn: 0.0134890	total: 1.14s	remaining: 1.57s
421:	learn: 0.0134872	total: 1.15s	remaining: 1.57s
422:	learn: 0.0134832	total: 1.15s	remaining: 1.57s
423:	learn: 0.0134822	total: 1.15s	remaining: 1.56s
424:	learn: 0.0134774	total: 1.15s	remaining: 1.56s
425:	learn: 0.0134724	total: 1.16s	remaining: 1.56s
426:	learn: 0.0134718	total: 1.16s	remaining: 1.55s
427:	learn: 0.0134671	total: 1.16s	remaining: 1.55s
428:	learn: 0.0134666	total: 1.16s	remaining: 1.55s
429:	learn: 0.0134661	total: 1.17s	remaining: 1.55s
430:	learn: 0.0134651	total: 1.17s	remaining: 1.54s
431:	learn: 0.0134614	total: 1.17s	remaining: 1.54s
432:	learn: 0.0134563	total: 1.17s	remaining: 1.54s
433:	learn: 0.0134554	total: 1.18s	remaining: 1.53s
434:	learn: 

603:	learn: 0.0129445	total: 1.64s	remaining: 1.08s
604:	learn: 0.0129423	total: 1.65s	remaining: 1.08s
605:	learn: 0.0129404	total: 1.65s	remaining: 1.07s
606:	learn: 0.0129396	total: 1.66s	remaining: 1.07s
607:	learn: 0.0129379	total: 1.66s	remaining: 1.07s
608:	learn: 0.0129368	total: 1.66s	remaining: 1.07s
609:	learn: 0.0129364	total: 1.67s	remaining: 1.06s
610:	learn: 0.0129351	total: 1.67s	remaining: 1.06s
611:	learn: 0.0129344	total: 1.67s	remaining: 1.06s
612:	learn: 0.0129319	total: 1.67s	remaining: 1.06s
613:	learn: 0.0129315	total: 1.68s	remaining: 1.05s
614:	learn: 0.0129278	total: 1.68s	remaining: 1.05s
615:	learn: 0.0129248	total: 1.68s	remaining: 1.05s
616:	learn: 0.0129237	total: 1.69s	remaining: 1.05s
617:	learn: 0.0129198	total: 1.69s	remaining: 1.04s
618:	learn: 0.0129072	total: 1.69s	remaining: 1.04s
619:	learn: 0.0129054	total: 1.69s	remaining: 1.04s
620:	learn: 0.0128958	total: 1.7s	remaining: 1.03s
621:	learn: 0.0128952	total: 1.7s	remaining: 1.03s
622:	learn: 0.

807:	learn: 0.0123679	total: 2.19s	remaining: 520ms
808:	learn: 0.0123678	total: 2.19s	remaining: 517ms
809:	learn: 0.0123549	total: 2.19s	remaining: 515ms
810:	learn: 0.0123455	total: 2.2s	remaining: 512ms
811:	learn: 0.0123439	total: 2.2s	remaining: 510ms
812:	learn: 0.0123423	total: 2.21s	remaining: 507ms
813:	learn: 0.0123273	total: 2.21s	remaining: 505ms
814:	learn: 0.0123260	total: 2.21s	remaining: 502ms
815:	learn: 0.0123257	total: 2.21s	remaining: 499ms
816:	learn: 0.0123240	total: 2.22s	remaining: 496ms
817:	learn: 0.0123226	total: 2.22s	remaining: 494ms
818:	learn: 0.0123222	total: 2.22s	remaining: 491ms
819:	learn: 0.0123178	total: 2.23s	remaining: 488ms
820:	learn: 0.0123168	total: 2.23s	remaining: 486ms
821:	learn: 0.0123155	total: 2.23s	remaining: 483ms
822:	learn: 0.0123114	total: 2.23s	remaining: 480ms
823:	learn: 0.0123089	total: 2.23s	remaining: 478ms
824:	learn: 0.0123078	total: 2.24s	remaining: 475ms
825:	learn: 0.0123066	total: 2.24s	remaining: 472ms
826:	learn: 0.

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.3s finished


In [7]:
res = resultado.sort_values(by='Train Score', ascending=False)
res

,Model,Window,Neutral,Train Score,Test Score,Params
2,LightGBM,4,0,-0.000277,-0.461229,"{'colsample_bytree': 0.8856577329473848, 'gamma': 3.6013544052084923, 'learning_rate': 0.3758676261960617, 'max_depth': 3, 'min_child_weight': 10.673440411307277, 'n_estimators': 88, 'reg_alpha': 6.731584675179195, 'subsample': 0.8985245815075779}"
2,LightGBM,3,1,-0.000277,-0.325904,"{'colsample_bytree': 0.9619452745919179, 'gamma': 4.322266919878999, 'learning_rate': 0.4372849271878025, 'max_depth': 4, 'min_child_weight': 2.3826881300886362, 'n_estimators': 98, 'reg_alpha': 3.4618205641436104, 'subsample': 0.9617979854298839}"
2,LightGBM,3,0,-0.000277,-0.260787,"{'colsample_bytree': 0.675311554952436, 'gamma': 5.724090943697377, 'learning_rate': 0.08995844451999406, 'max_depth': 20, 'min_child_weight': 5.473519799567251, 'n_estimators': 74, 'reg_alpha': 2.482836502732961, 'subsample': 0.7781901227653004}"
2,LightGBM,4,1,-0.000279,-0.558599,"{'colsample_bytree': 0.9816830327385575, 'gamma': 2.1614640725552814, 'learning_rate': 0.03148430307326366, 'max_depth': 21, 'min_child_weight': 7.658865556913343, 'n_estimators': 53, 'reg_alpha': 4.9865655114693, 'subsample': 0.7352285899164377}"
2,LightGBM,5,1,-0.000279,-0.375260,"{'colsample_bytree': 0.7995241815874387, 'gamma': 4.09841608285314, 'learning_rate': 0.23485216513788348, 'max_depth': 8, 'min_child_weight': 20.199553484934157, 'n_estimators': 82, 'reg_alpha': 8.4247944120769, 'subsample': 0.9431679381210523}"
2,LightGBM,5,0,-0.000282,-0.302055,"{'colsample_bytree': 0.9680016518579571, 'gamma': 4.0065626525615645, 'learning_rate': 0.2349576122955992, 'max_depth': 16, 'min_child_weight': 4.351419460774261, 'n_estimators': 35, 'reg_alpha': 4.411474195496665, 'subsample': 0.952977629679902}"
2,LightGBM,1,0,-0.000316,-0.143681,"{'colsample_bytree': 0.9449186502030844, 'gamma': 5.718483974224489, 'learning_rate': 0.37070523796134336, 'max_depth': 13, 'min_child_weight': 18.78649412119168, 'n_estimators': 75, 'reg_alpha': 1.8460441233883473, 'subsample': 0.8361212951185109}"
2,LightGBM,2,1,-0.000316,-0.163969,"{'colsample_bytree': 0.8800351982692248, 'gamma': 8.519044009289603, 'learning_rate': 0.4812389044078623, 'max_depth': 15, 'min_child_weight': 5.306659659140415, 'n_estimators': 22, 'reg_alpha': 4.706846773651585, 'subsample': 0.8828007235721198}"
2,LightGBM,1,1,-0.000316,-0.165472,"{'colsample_bytree': 0.9709163308132353, 'gamma': 4.769129582442653, 'learning_rate': 0.23997079481308997, 'max_depth': 24, 'min_child_weight': 9.932488759611548, 'n_estimators': 58, 'reg_alpha': 2.919538051831611, 'subsample': 0.866940256501805}"
2,LightGBM,2,0,-0.000317,-0.318279,"{'colsample_bytree': 0.7791286869540331, 'gamma': 4.789807174708653, 'learning_rate': 0.4279469488456941, 'max_depth': 20, 'min_child_weight': 12.800984666129029, 'n_estimators': 63, 'reg_alpha': 2.9937701395008727, 'subsample': 0.9349511987978549}"


In [8]:
res.to_csv('resultados2.csv')